In [2]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
from torch.nn import BCEWithLogitsLoss

In [3]:
# Bước 1: Đọc và tiền xử lý dữ liệu
data_path = "facebook_comment_2k7.csv"
data = pd.read_csv(data_path)
data['label'] = data['label'].apply(lambda x: eval(x))

In [4]:
# Bước 2: Tokenization
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
encoded_data = tokenizer.batch_encode_plus(
    data['text'].tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=256, 
    truncation=True,
    return_tensors='pt'
)

In [5]:
# Bước 3: Chuẩn bị dữ liệu
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(data['label'])
labels_tensor = torch.tensor(labels, dtype=torch.float)

# Tạo TensorDataset và DataLoader
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels_tensor)

# Chia dữ liệu thành tập huấn luyện và kiểm thử
train_dataset, val_dataset = train_test_split(dataset, test_size=0.1)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [6]:
train_loader

In [5]:
# Bước 4: Xây dựng mô hình
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(mlb.classes_))
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(mlb.classes_))
model.load_state_dict(torch.load('model.pth'))
model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [10]:
# Bước 5: Huấn luyện mô hình
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

epochs = 4

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = tuple(b.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")) for b in batch)
        inputs, masks, labels = batch
        model.zero_grad()
        
        outputs = model(inputs, attention_mask=masks)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        total_loss += loss.item()
        
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [7]:
correct_predictions = 0
total_samples = 0

for i, batch in enumerate(val_loader):
    inputs, masks, labels = batch
    model.eval()
    with torch.no_grad():
        outputs = model(inputs, attention_mask=masks)
        predictions = torch.sigmoid(outputs.logits)
        # print(predictions)
        predicted_labels = (predictions > 0.3).int()
        predicted_labels = mlb.inverse_transform(predicted_labels)

    true_labels_list = mlb.inverse_transform(labels.int().cpu().numpy())

    for pred_labels, true_labels in zip(predicted_labels, true_labels_list):
        if set(pred_labels) == set(true_labels):
            correct_predictions += 1
        total_samples += 1

    for j in range(inputs.size(0)):  # Duyệt qua từng mẫu trong batch
        text = tokenizer.decode(inputs[j], skip_special_tokens=True)
        print("Sequence:", text)
        print("Predicted label:", predicted_labels[j])
        print("Hand-labeled:", mlb.inverse_transform(labels)[j])

Sequence: Dm~ đồ ranh con dám hun ng thg của ta huhu
Predicted label: ('hate_speech',)
Hand-labeled: ('hate_speech',)
Sequence: Chúng em đang còn sin tint Merzy có chất son và tone màu son tương tự, em gửi chị tham khảo qua nhé ạ
Predicted label: ('normal',)
Hand-labeled: ('normal',)
Sequence: Có con mắt j của sasuke có khả năng điều khiển á. Lấy cn mắt đó đem chịch gái khỏi chê :)
Predicted label: ('sexaully_explixit',)
Hand-labeled: ('sexaully_explixit',)
Sequence: Cần tiền vay được ngayNói không với phí trước VAY NHANH TRONG NGÀY CHỈ CẦN CMT+ATMmiễn phí lãi 10 ngày đầu tiênAi cần vay đăng ký tại đây https://shorten.asia/29GChqeN
Predicted label: ('dangerous_content',)
Hand-labeled: ('dangerous_content',)
Sequence: Em sinh e be xong nguc phang chang co ji! E muon lam lang nguc,nhung nguc ben phai cua e co mot u xo nho bang hat dau,neu lam pho thuat co anh huong gi ko bs?,xin tu van gium e!
Predicted label: ('normal',)
Hand-labeled: ('normal',)
Sequence: Nguyễn Anh Tú im cái j cá

In [11]:
accuracy = correct_predictions / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.8481481481481481


In [7]:
# At least one of the predicted labels was true
accuracy = correct_predictions / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.9851851851851852


In [8]:
# Nhập chuỗi từ bàn phím
input_text = "mày ngu quá"

# Tokenization
encoded_input = tokenizer.encode_plus(
    input_text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=256, 
    truncation=True,
    return_tensors='pt'
)

input_ids = encoded_input['input_ids'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
attention_mask = encoded_input['attention_mask'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Dự đoán
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.sigmoid(outputs.logits)
    predicted_labels = (predictions > 0.5).int()
    predicted_labels = mlb.inverse_transform(predicted_labels)

# In chuỗi và nhãn dự đoán
print("Chuỗi bạn nhập:", input_text)
print("Nhãn dự đoán:", predicted_labels[0])

Chuỗi bạn nhập: mày ngu quá
Nhãn dự đoán: ('harassment', 'hate_speech')


In [9]:
# Nhập chuỗi từ bàn phím
input_text = "sao mày ngu thế con đĩ"

# Tokenization
encoded_input = tokenizer.encode_plus(
    input_text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=256, 
    truncation=True,
    return_tensors='pt'
)

input_ids = encoded_input['input_ids'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
attention_mask = encoded_input['attention_mask'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Dự đoán
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.sigmoid(outputs.logits)
    predicted_labels = (predictions > 0.5).int()
    predicted_labels = mlb.inverse_transform(predicted_labels)

# In chuỗi và nhãn dự đoán
print("Chuỗi bạn nhập:", input_text)
print("Nhãn dự đoán:", predicted_labels[0])

Chuỗi bạn nhập: sao mày ngu thế con đĩ
Nhãn dự đoán: ('harassment', 'hate_speech')


In [8]:
# Nhập chuỗi từ bàn phím
input_text = "hiếp dâm ẩm thực"

# Tokenization
encoded_input = tokenizer.encode_plus(
    input_text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=256, 
    truncation=True,
    return_tensors='pt'
)

input_ids = encoded_input['input_ids'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
attention_mask = encoded_input['attention_mask'].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Dự đoán
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.sigmoid(outputs.logits)
    predicted_labels = (predictions > 0.5).int()
    predicted_labels = mlb.inverse_transform(predicted_labels)

# In chuỗi và nhãn dự đoán
print("Chuỗi bạn nhập:", input_text)
print("Nhãn dự đoán:", predicted_labels[0])

Chuỗi bạn nhập: hiếp dâm ẩm thực
Nhãn dự đoán: ('sexaully_explixit',)
